# Freeform explorer and column explorer

A function `freeform()` to list the freeforms (open-ended story tags) associated with a character pair doing slash.

Also a more general function `mentions()` to search a column, and the report the frequences in another column of those matching rows.

## Load dataframe from .CSV into `df`

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt

import ds9
df = ds9.df()

## Define the data explorer for freeforms

This code is easily enough understood if you have src/misc/pandas-cheetsheet.md handy, as it's mainly phrases from that.

In [2]:
class Explorer:
    def __init__(self, df):
        # Narrow the dataframe to just the columns of interest
        self.df = df[['relationshipspairslash',
                      'freeforms',
                      'categories',
                      'charactersclean',
                      'fandoms',
                      'relationshipspair',
                      'relationshipspairamp',
                      'relationshipspax',
                      'relationshipspaxamp',
                      'relationshipspaxslash']]
        
    def freeforms(self, pair, top=0):
        # Narrow the dataframe to `pair_df` which contains just the rows which contain the pairing
        pair_df = pd.DataFrame()
        for index, relationshipspairslash in zip(self.df.index, self.df['relationshipspairslash']):
            if pair in relationshipspairslash:
                # Match found, copy the whole row to the bottom of the new dataframe
                row = self.df[self.df.index == index].copy()
                pair_df = pd.concat([pair_df, row])
            
        # Explode out the freeforms list into separate rows in a dataframe `explode_df`
        explode_df = pd.DataFrame()
        for index, freeforms in zip(pair_df.index, pair_df['freeforms']):
            for i in freeforms:
                row = pair_df[pair_df.index == index].copy()
                row['freeforms'] = i
                explode_df = pd.concat([explode_df, row], ignore_index=True)

        # Create a dataframe of (freeforms, freq)
        freq_df = explode_df['freeforms'].value_counts()
        # Narrow the returned dataframe to the N most popular if that was requested
        if top > 0:
            freq_df = freq_df.head(top)
        
        return freq_df

    
    # A more general version of freeforms() above.
    def mentions(self, phrase, search='relationshipspairslash', collect='relationshipspairslash'):
        # Search for `phrase` in dataset column `search`
        match_df = pd.DataFrame()
        for index, value in zip(self.df.index, self.df[search]):
            if phrase in value:
                row = self.df[self.df.index == index].copy()
                match_df = pd.concat([match_df, row])
        # Explode column `collect`
        explode_df = pd.DataFrame()
        for index, values in zip(match_df.index, match_df[collect]):
            for i in values:
                row = match_df[match_df.index == index].copy()
                row[collect] = i
                explode_df = pd.concat([explode_df, row], ignore_index=True)
        # Count frequencies
        freq_df = explode_df[collect].value_counts()
        return freq_df

## Using the explorer

### Freeform explorer
 
Run and save to variable `f` the results of exploring the freeform text 'Elim Garak/Julian Bashir'

```
ex = Explorer(df)
f = ex.freeforms('Elim Garak/Julian Bashir')
```
 
Print that exploration
 
```
f
```
 
Save that exploration to a .CSV file. CSV files don't have any internal documentation, so take care with the file name.

```
f.to_csv('freeform-explorer-garak-bashir.csv')
```

You don't need to restart Jupyter to explore another pairing, just run the `freeform()` function again. We might only want the top 10 most popular freeforms:

```
f = ex.freeforms('Elim Garak/Julian Bashir', top=10)
```

or to explore another pairing entirely:

```
f = ex.freeforms('Jadzia Dax/Kira Nerys')
```
 
Similarly you don't need to always save the results to `f`. Any variable name is fine.


### List field explorer

For fields which are lists the explorer can `search` one column for the presence of the value. Those matching
rows then have the `collect` column summed for each of the list values in that column.

The is a generalised version of `freeforms()`. The same work as `ex.freeforms('Jadzia Dax/Kira Nerys')` is
done with `ex.mentions('Jadzia Dax/Kira Nerys', search='relationshipspairslash', collect='freeforms')`.

```
m = ex.mentions('Elim Garak/Julian Bashir', search='relationshipspairslash', collect='charactersclean')
m
```

The column names which can be given into `search=` and `collect=` are:

* `'relationshipspairslash'`
* `'freeforms'`
* `'categories'`
* `'charactersclean'`
* `'fandoms'`
* `'relationshipspair'`
* `'relationshipspairamp'`
* `'relationshipspax'`
* `'relationshipspaxamp'`
* `'relationshipspaxslash'`

Plus all those names again, ending with `gender` and `species`.

There is a bug at the moment. If there is no match then the program crashes. I'll fix that.


### Load the explorer

In [3]:
ex = Explorer(df)

### Try one

Here's some recent exploration:

```
f = ex.freeforms('Odo/Quark', top=20)
f
f.to_csv('freeforms-filename.csv')
```

```
m = ex.mentions('Elim Garak/Julian Bashir', search='relationshipspairslash', collect='charactersclean')
m
m.to_csv('mentions-filename.csv')
```

```
ex.freeforms('Ezri Dax/Julian Bashir', top=40)

```

```
ex.mentions('Elim Garak/Tora Ziyal', search='relationshipspairslash', collect='relationshipspairslash')
```

```
ex.mentions('Elim Garak & Tora Ziyal', search='relationshipspairamp', collect='relationshipspairamp')
```

```
ex.mentions('Ezri Dax', search='charactersclean', collect='charactersclean')
```

```
ex.mentions('Jadzia Dax', search='charactersclean', collect='charactersclean')
```

Here's a complex one which is handy if you want to do and record a lot of similar searches. Set up the variables `search` and `collect` then \[Kernel\] | \[Restart & Run All\]. Then keep changing `name` and pressing \[Run\].

```
name = 'Jadzia Dax'
search = 'charactersclean'
collect = 'charactersclean'
data = ex.mentions(name, search=search, collect=collect)
filename = ds9.csvfilename('freeform-explorer', search, collect, name)
data.to_csv(filename)
print('freeform-explorer: {}, {}, {}'.format(name, search, collect))
print('Saved to {}'.format(filename))
data
```

Try one below...

Copy one of the examples above, then press \[Run\] to re-run the changed Jupyter cell.